In [1]:
# pip install ipympl

In [1]:
#Initial Example Code is Pasted Below

import requests



# Define the headers to send to the API URI
headers = {
    'content-type':'application/x-www-form-urlencoded;',
    'Access-Control-Allow-Origin':'*',
#    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:27.0) Gecko/20100101 Firefox/27.
}

#Compound can be a list of strings, be sure to use empirical formula i.e. "Au", not "Gold"
compound = ["Cu", "Sn"]
#x-axis variable, use code for in decodre table, not name of the variable i.e. Temperature: "T"
var = "T"
#property attempting to measure, use code in decoder table, not name i.e. Electrical Resisitivity: "ER"
prop = "ER"
#representation of data, use code in decoder table, i.e. Direct: "A", if no representation wanted, use bool False
rep = False
#state of matter for data, use code in decoder table, i.e. Crystal: "C", if all states wanted, use bool False
state = False

# Define the search data JSON to send to the API URI
search_data = {"property_search_code":prop}
# search_data = {"author_last_name":['Wilthan']}

#"exclude_all":"True", "required_compounds":compound, "property_search_code":"VDN"

# Define the URL to send the request
# Note that this example does require an authentication key to get the data. The one given here is an invalid authkey and will need to be replaced by
# a valid authentication key which may be requested free of charge fapi_key = open('C:/Users/bmd4/Desktop/API Work/TRC-Alloys API Key.txt').read()r# om TRCalloy@nist.gov.

api_key = open('D:/Bryan/Desktop/NIST/TRC-Alloys API Key.txt').read()
# api_key = open('C:/Users/bmd4/Desktop/API Work/TRC-Alloys API Key.txt').read()
url = f'http://trcsrv2.boulder.nist.gov/MetalsAlloyAPI/search?authkey={api_key}'
# url=f'https://trc.nist.gov/MetalsAlloyAPI/search?authkey={api_key}'

In [3]:
import re
# Get the compound data from a requests post
compound_data_response = requests.post(url, json=search_data, headers=headers)
# The compound_data object is a "response" object and the actual output JSON is in the
# text field of that object. So use compound_data_response.text to use the actual output JSON
with open('Test.json', 'w') as fp:
    fp.write((re.sub(r'[^\x00-\x7f]',r'', compound_data_response.text)))

In [2]:
import json

with open('Test.json') as json_file:
    data = json.load(json_file)

with open('decoder_table.json') as json_file:
    decoder_table = json.load(json_file)

In [3]:
from collections import Counter
element_ids = []
for a in data['TRC_data']:
    for b in a['compounds']:
        for c in compound:
            if b['formula_hill'] == c:
                c = [b['compound_id']]
                if c[0] not in element_ids:
                    element_ids.append(c[0])
print(element_ids)

[7671, 10884]


In [4]:
temp_x = "N/A"
temp_y = "N/A"
x_ax = []
y_ax = []
x_axt = []
y_axt = []
datasets = []
phase = []
num1 = 0
num2 = 0
num3 = 0
num4 = 0
num5 = 0

for a in data['TRC_data']:
    for b in a['systems']:
        try:
            if (Counter(b['compound_ids']) == Counter(element_ids)):
                for s in b['phases']:
                    if (s['type'] == state):
                        phase.append(s['phase_id'])
                num3 += 1
                for c in b['data_sets']:
                    num5 += 1
                    for v in c['variables']:
                        if v['variable_name'] == prop and (v['representation'] == rep or rep == False):
                            num1 += 1
                            temp_y = v['variable_id']
                        if v['variable_name'] == var:
                            num2 += 1
                            temp_x = v['variable_id']
                    v_num = 0
                    num = 0
                    for p in c['states']:
                        if p['phase_id'] in phase:
                            v_num += 1
                        num += 1
                    for d in c['data']:
                        if (str(temp_x) in str(c['data'])) and (str(temp_y) in str(c['data'])) and (v_num == num or state == False):
                            num4 += 1
                            if c['data_set_id'] not in datasets:
                                datasets.append(c['data_set_id'])
                            if d['variable_id'] == temp_y:
                                for e in d['data_values']:
                                    y_axt.append(e['value'])
                                y_ax.append(y_axt)
                                y_axt = []
                            if d['variable_id'] == temp_x:
                                for e in d['data_values']:
                                    x_axt.append(e['value'])
                                x_ax.append(x_axt)
                                x_axt = []
        except KeyError:
            #print(a['citation'])
            break

print(num3, num1, num2, num4, num5)
print(datasets)
print(phase)

11 16 22 59 26
[2246, 6944, 7785, 8505, 8506, 14345, 18706, 20205, 20206, 20952, 23277, 23278, 25156, 25157, 25158, 25159]
[]


In [5]:
for a in decoder_table['PRP']:
    if a == var:
        x_lab = decoder_table['PRP'][var]
    if a == prop:
        y_lab = decoder_table['PRP'][prop]
for b in decoder_table['UnitsStandard']:
    if b == var:
        x_un = decoder_table['UnitsStandard'][var]
    if b == prop:
        y_un = decoder_table['UnitsStandard'][prop]


In [6]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math
%matplotlib widget

x = []
y = []

print(len(x_ax),len(y_ax))

adj = math.ceil(len(y_ax)/20)

print(adj)

# print(x_ax)
# print(y_ax)
figure(figsize = (8 + adj, 5))

for i,j in zip(x_ax, y_ax):
    if len(i) != len(j):
        for k in range(1,len(j)):
            i.append(i[0])
    for m,n in zip(i,j):
        if (type(m) == int or type(m) == float) and (type(n) == int or type(n) == float) and n != 0:
            x.append(m)
            y.append(n)
    sc = plt.scatter(x,y)
    x = []
    y = []

plt.legend(datasets, ncol = adj, bbox_to_anchor=(1.04,1), loc="upper left", borderaxespad=0) #
plt.title(compound)
plt.xlabel(f'{x_lab}: {x_un}')
plt.ylabel(f'{y_lab}: {y_un}')
plt.tight_layout()
plt.show

16 16
1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<function matplotlib.pyplot.show(block=None)>